In [ ]:
import unittest

In [ ]:
# For customers
ETHERSCAN_API_KEY = 'YOUR_OWN_KEY' 
BSCSCAN_API_KEY = 'YOUR_OWN_KEY'
CONNECTION_STRING = 'https://mainnet.infura.io/v3/YOUR_OWN_KEY'

In [ ]:
# Only for test
ETHERSCAN_API_KEY = 'EWYSGQ5PEK84PRCNSCKHHZWJIWWKYYABCE' 
BSCSCAN_API_KEY = 'I6ZZ57GWAS39M85QC4AJFC43Y5DUA8IU8I'
#CONNECTION_STRING = 'https://mainnet.infura.io/v3/b8932e6edfa1498ab41ce07e5686c2b4'
CONNECTION_STRING = 'https://mainnet.infura.io/v3/40816bb42ce34f808b4afe219e9d41cb'
TESTADDRESS = '0x70e36f6bf80a52b3b46b3af8e106cc0ed743e8e4' 
_contractAddress = TESTADDRESS
_apiKey = ETHERSCAN_API_KEY
_connectionString = CONNECTION_STRING

In [ ]:
# Test web3 connection
from classes import blockchain_v1_0_0_0 as blockchain
blockchain = blockchain.blockchain()
blockchain.blockchainConnection

In [ ]:
blockchain.inizializeSmartContracts(ETHERSCAN_API_KEY, CONNECTION_STRING)

In [2]:

if True: True else: False

SyntaxError: invalid syntax (<ipython-input-2-dcc01bfea0cd>, line 1)

In [ ]:
blockchain.downloadContractLogsContent(_contractAddress, _apiKey, _connectionString)

In [ ]:
from classes import bitQuery_v1_0_0_0 as bitQuery
import json
bitQuery = bitQuery.bitQuery()

In [ ]:
bitQuery.getLogs(apiKey)

In [ ]:
util = util.utilities()

In [ ]:
info = util.getFrameworkInfo()

In [ ]:
apiKey = 'BQYvYOBEZWH2ymkvKTC3tbDRgprmFeOY'

In [ ]:
info['Versioning']['version']

In [ ]:
import pathlib
from os import path, listdir
import os
import configparser
import json


# Reading Configs
config = configparser.ConfigParser()
print("{0}/config.ini".format(pathlib.Path(__file__).parent.parentòknrfabsolute()))
#config.read("{0}/config.ini".format(pathlib.Path(__file__).parent.absolute()))

In [ ]:
exec(compile(open('classes/libraries_v1_0_0_0.py').read()))

In [ ]:
from past.builtins import execfile

execfile('classes/libraries_v1_0_0_0.py')

In [ ]:
pd.DataFrame()

In [ ]:
blockchain = lib.blockchain()

In [ ]:
from graphene import ObjectType, String, Schema

class Query(ObjectType):
    # this defines a Field `hello` in our Schema with a single Argument `name`
    # note: DEFINE THE DATATYPE
    hello = String(name=String(default_value="stranger"))
    goodbye = String()

    # our Resolver method takes the GraphQL context (root, info) as well as
    # Argument (name) for the Field and returns data for the query Response
    def resolve_hello(root, info, name):
        return f'Hello {name}!'

    def resolve_goodbye(root, info):
        return 'See ya!'

schema = Schema(query=Query)

In [ ]:
from graphene import ObjectType, String, Schema

In [ ]:
query = """
        {
  ethereum {
    dexTrades(options: {limit: 100, desc: "count"}) {
      count
      protocol
    }
  }
}
"""

In [ ]:
from graphene import ObjectType, String, Schema

class Query(ObjectType):
    getDexTradeByProtocol = String(
        count = String(),
        protocol = String()
    )

    def resolve_getDexTradeByProtocol(root, info):
        query = """
                {
          ethereum {
            dexTrades(options: {limit: 100, desc: "count"}) {
              count
              protocol
            }
          }
        }
        """
        return query

In [ ]:
schema = Schema(query=Query)

In [ ]:
print(schema)

In [ ]:
query_string = '{ getDexTradeByProtocol }'
result = schema.execute(query_string)
print(result)

In [ ]:
# we can query for our field (with the default argument)
query_string = '{ hello }'
result = schema.execute(query_string)
print(result.data['hello'])
# "Hello stranger!"

# or passing the argument in the query
query_with_argument = '{ hello(name: "GraphQL") },'
result = schema.execute(query_with_argument)
print(result.data['hello'])
# "Hello GraphQL!"

In [ ]:
# we can query for our field (with the default argument)
query_string = '{ goodbye }'
resesult = schema.execute(query_with_argument)
print(result.data['goodbye'])
# "Hello GraphQL!"

In [ ]:
dir(bitQuery)

In [ ]:
from classes import bitQuery_v1_0_0_0 as bitQueryClass

In [ ]:
apiKey = 'BQYvYOBEZWH2ymkvKTC3tbDRgprmFeOY'

In [ ]:
bitQuery = bitQueryClass.bitQuery()
query = bitQueryClass.query()

In [ ]:
data = bitQuery.directQuery(query.dexTradeByProtocol(), apiKey)

In [ ]:
print(data)

In [ ]:
bitQuery.getValueFromGraphQL(data, ['count', 'protocol'])

In [ ]:
query = """
query{
  bitcoin{
    blocks{
      count
    }
   }
}
"""

In [ ]:
data = bitQuery.directQuery(query, apiKey)
print(data)
bitQuery.getValueFromGraphQL(data, ['count'])

In [ ]:
import json

In [ ]:
data

In [ ]:
import re

In [ ]:
data = str(data)

In [ ]:
data

In [ ]:
for match_start in re.finditer('r(count)', data):
    print('hey')

In [ ]:
a = [1] * len([1, 2])

In [ ]:
a[0]

In [ ]:
5 % 3

In [ ]:
import pandas as pd
def getValueFromGraphQL(data, _nameList):
    result = []
    for name in _nameList:
        for match_start in re.finditer(r'({0})'.format(name), data):
            tmp = data[match_start.end()+3:len(data)-match_start.end()-3]
            firstComma = re.search(', ', tmp)
            if(firstComma != None):
                firstComma = firstComma.start()
                value = data[match_start.end()+3:match_start.end()+3+firstComma].replace('}','').replace("'","") #deleting special chars
                result.append(value)
    
    # Creating DataFrame with result
    df = pd.DataFrame(columns = cols)
    tmpDf = df
    lenghtNameList = len(_nameList)
    iterableItems = len(result) // lenghtNameList - 1
    print(iterableItems)
    for i in range(0,iterableItems):
        data = []
        # Concatenate Values
        for j in range(0, lenghtNameList):
            data.append(result[j * iterableItems + i + 1])
        print(data)
        tmpDf = pd.DataFrame([data], columns = cols)
        df = pd.concat([df, tmpDf])
    return df

In [ ]:
getValueFromGraphQL(data, ['count','protocol'])

In [ ]:
a.concat(b)

In [ ]:
b

In [ ]:
cols = ['count', 'protocol']
df = pd.DataFrame(columns = cols)
tmpDf = df

In [ ]:
for i in range(0,len(a)):
    
    tmpDf = pd.DataFrame([[a[i], b[i]]], columns = cols)
    df = pd.concat([df, tmpDf])


In [ ]:
data.to_dict()

In [ ]:
df

In [ ]:
import datetime    

start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2016, 1, 27)

In [ ]:
start

In [ ]:
from classes import utilities_v1_0_0_0 as utilities
util = utilities.utilities()
info = util.getFrameworkInfo()

In [ ]:
print(info['Resources']['v1_0_0_0_resource_link'])

In [2]:
from time import sleep
from tqdm import tqdm
for i in tqdm(range(10)):
    sleep(3)

100%|██████████| 10/10 [00:30<00:00,  3.00s/it]


In [ ]:
from classes import unitTest_v1_0_0_0 as blockchain

class blockchain_v1_0_0_0_unitTest():

    def ___init__(self, name):
        self.name = name
        
    def test_web3Connection_infura(self):
        
        blockchain = blockchain.blockchain()
        INFURA_ID = '40816bb42ce34f808b4afe219e9d41cb' #change it with your own id
        connectionString = 'https://mainnet.infura.io/v3/{0}'.format(INFURA_ID)

        self.assertEqual(blockchain.blockchainConnection(connectionString).isConnected(), True)


In [ ]:
# Only for develop
ETHERSCAN_API_KEY = 'EWYSGQ5PEK84PRCNSCKHHZWJIWWKYYABCE' 
BSCSCAN_API_KEY = 'I6ZZ57GWAS39M85QC4AJFC43Y5DUA8IU8I'
CONNECTION_STRING = 'https://mainnet.infura.io/v3/40816bb42ce34f808b4afe219e9d41cb'
TESTADDRESS = '0x70e36f6bf80a52b3b46b3af8e106cc0ed743e8e4' #COMPOUND
TXHASHTEST = '0x7a4e0d863a1c8da97bf94bc8880d61b3f716bb10c3961f46b1290840097cb175' # ON BINANCE

In [ ]:
from classes import blockchain_v1_0_0_0 as blockchain
import pandas as pd
blockchain = blockchain.blockchain()
dynamicClassFolder = 'classes/dynamicClass'
#_smartContractAddress = '0x70e36f6bf80a52b3b46b3af8e106cc0ed743e8e4'
fileName = 'dynamic_events.py'
file = open("{0}/{1}".format(dynamicClassFolder, fileName), 'w')

dfSmartContract = pd.read_csv('data/smartContract.csv')
for indexSmartContract in range(0, len(dfSmartContract)):
    (nameAddress, address) = (dfSmartContract['NameAddress'][indexSmartContract], dfSmartContract['Address'][indexSmartContract])
    (smartContract, ABI) = blockchain.getContract(address, ETHERSCAN_API_KEY, CONNECTION_STRING)   
    blockchain.saveContractData(address, smartContract, ABI) #save events and functions names on data/contracts/{address}
    dfEvents = pd.read_csv('data/contracts/{0}/events.csv'.format(address))
    if(len(dfEvents)>0):
        file.writelines("\n\nif(contractName == '{0}'): # {1}\n\n".format(address.lower(), nameAddress))
        for indexEvents in range(0, len(dfEvents)):
            file.writelines("    eventList.append(_contract.events.{0}()):\n".format(dfEvents['event'][indexEvents]))

#ADD THE SAME LOGIC TO FUNCTIONS
file.close() 

In [ ]:
# Run by terminal if these modules are not in your enviroment
# pip install pandas-datareader
# pip3 install tensorflow
# pip install numpy
# pip install pandas
# pip install matplotlib
# pip install scipy
# pip install -U scikit-learn
# pip install keras
# pip install get-all-tickers
# pip install "graphene>=2.0"

# Web3 libraries

from web3 import Web3

# Data Transformation/Analysis libraries

import pandas as pd
import numpy as np
import pandasql as psql
import string
import re
import math

# Data Reader libraries

import pandas_datareader as web
import requests

# Models libraries

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

# OS libraries

from datetime import datetime
import os
from os import listdir
from os import path
from os.path import isfile, join, isdir
from datetime import datetime
import sys

# Process libraries

import warnings

# Library default Settings

plt.style.use('fivethirtyeight')

In [ ]:
CONTRACT_FOLDER = 'data/contracts'
contractList = [f for f in listdir(CONTRACT_FOLDER) if isdir(join(CONTRACT_FOLDER, f))]

eventPrefix = 'event_'
blockNumberDf = pd.DataFrame()
dfList = {}
print(contractList)
for contractFolder in contractList:
    if(contractFolder != '.ipynb_checkpoints'):
        cFolder = CONTRACT_FOLDER+'/'+contractFolder
        eventsDataList = [f for f in listdir(cFolder) if isfile(join(cFolder, f))]
        print(eventsDataList)
        for eventsData in eventsDataList:
            if(eventsData[0:len(eventPrefix)] == eventPrefix):
                tmpDf = pd.read_csv(cFolder+'/'+eventsData)
                indexKey = 0
                foundKey = 0
                for key in tmpDf.keys():
                    if(key == 'blockHash'):
                        foundKey = indexKey
                    indexKey = indexKey + 1
                # KEYNAME EXAMPLE: A6F6BF_CASHPRIOR. A = ADDRESS, 6F6BF = LAST 5 DIGITS OF THE ADDRESS, CASHPRIOR = EVENT NAME
                keyName = 'A'+contractFolder.replace(" ", "_")[len(contractFolder)-5:len(contractFolder)]+"_"+eventsData[len(eventPrefix):len(eventsData)-20] 
                keyName = keyName.upper()
                dfList[keyName] = tmpDf.iloc[:, 1:foundKey]
                dfList[keyName]['blockNumber'] = tmpDf['blockNumber']
                blockNumberDf = pd.concat([blockNumberDf, tmpDf['blockNumber']])

blockNumberList = pd.DataFrame(blockNumberDf[0].unique(), columns = ['blockNumber'])

sql = "SELECT {0} FROM blockNumberList A "
colsBase = 'A.blockNumber, '
cols = colsBase
print(dfList.keys())
for key in dfList.keys():
    groupCols = []
    afterCols = []
    for keyColumn in dfList[key].keys():
        if(keyColumn == 'from'): #special word
            dfList[key].rename(columns={'from':'fromAddress'}, inplace=True)
            keyColumn = 'fromAddress'
        if(keyColumn == 'to'): #special word
            dfList[key].rename(columns={'to':'toAddress'}, inplace=True)
            keyColumn = 'toAddress'
        if(keyColumn != 'blockNumber'):
            groupCols.append('MAX('+keyColumn+') AS '+key+'_'+keyColumn+',')
            afterCols.append(key+'_'+keyColumn+', ')
    selectGroup = ' '.join(groupCols)
    selectGroupBase = ' '.join(afterCols)
    cols = cols + selectGroup

    df = dfList[key]
    joinStatement = "LEFT JOIN df B on A.blockNumber = B.blockNumber "
    groupByStatement = "GROUP BY A.blockNumber ORDER BY A.blockNumber"
    finalSql = sql.format(cols[0:len(cols)-1])+joinStatement+groupByStatement
    colsBase = colsBase + selectGroupBase
    cols = colsBase
    print("{0} event was imported".format(key))
    blockNumberList = psql.sqldf(finalSql)

dt = datetime.now().strftime("%Y%m%d_%H%M%S")
blockNumberList.to_csv('{0}/smartContractsLogsGrouped_{1}.csv'.format(CONTRACT_FOLDER, dt))
print('File saved on {0}/smartContractsLogsGrouped_{1}.csv'.format(CONTRACT_FOLDER, dt))

In [1]:
from classes import listeners_v1_0_0_0 as listeners
listeners = listeners.listeners()
listeners.newListener('ID', ['test'], [5], 2)

KeyError: 'self.__test'

In [ ]:
from tqdm.notebook import tqdm
from time import sleep
pbar = tqdm(range(100), leave=True)
for i in range(100):
    pbar.write('processed: {}'.format(i))
    pbar.update(1)
    sleep(1)









                        







 90%|█████████ | 90/100 [03:46<00:25,  2.52s/it]

processed: 0










                        







 90%|█████████ | 90/100 [03:47<00:25,  2.53s/it]

processed: 1










                        







 90%|█████████ | 90/100 [03:48<00:25,  2.54s/it]

...

processed: 5










                        







 90%|█████████ | 90/100 [03:52<00:25,  2.59s/it]

processed: 6










                        







 90%|█████████ | 90/100 [03:53<00:25,  2.60s/it]

processed: 7


In [1]:
from classes import terraChain_v1_0_0_0 as terraChain
terraChain = terraChain.terraChain()
terra = terraChain.connect()
print(terra.tendermint.node_info())

{'node_info': {'protocol_version': {'p2p': '7', 'block': '10', 'app': '0'}, 'id': 'f472dfd594de250acce7ce65c85ec2936f291377', 'listen_addr': 'tcp://0.0.0.0:26656', 'network': 'columbus-4', 'version': '0.33.9', 'channels': '4020212223303800', 'moniker': 'rpc4', 'other': {'tx_index': 'on', 'rpc_address': 'tcp://10.0.30.45:26657'}}, 'application_version': {'name': 'terra', 'server_name': 'terrad', 'client_name': 'terracli', 'version': '0.4.2-rc0-25-gb53f1bc', 'commit': 'b53f1bc72f018806f62b8770256902721b8e499d', 'build_tags': 'netgo ledger muslc,', 'go': 'go version go1.14.4 linux/amd64', 'build_deps': ['github.com/99designs/keyring@v1.1.6', 'github.com/ChainSafe/go-schnorrkel@v0.0.0-20200405005733-88cbf1b4c40d', 'github.com/CosmWasm/go-cosmwasm@v0.10.0 => github.com/terra-project/go-cosmwasm@v0.10.4', 'github.com/Workiva/go-datastructures@v1.0.52', 'github.com/bartekn/go-bip39@v0.0.0-20171116152956-a05967ea095d', 'github.com/beorn7/perks@v1.0.1', 'github.com/bgentry/speakeasy@v0.1.0', 'g

In [2]:
terra.tendermint.block_info()['block']['header']['height']

'3919016'

In [4]:
result = terra.wasm.contract_query('terra14z56l0fp2lsf86zy3hty2z47ezkhnthtr9yq76', {"balance": {}})
print(result)

LCDResponseError: Status 500 - contract query failed: parsing cw20_base::msg::QueryMsg: missing field `address`